In [1]:
import os

import _dunsros as DR

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from tqdm import tqdm
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split

from kan import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchmetrics as tm

device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps


In [ ]:
path = os.getcwd()
files = os.listdir(path + "/1")
files_xls = [f for f in files if f[-4:] == 'xlsx']
data_ = pd.DataFrame()
data = pd.DataFrame()
for f in files_xls:
    data_ = pd.read_excel("1/"+f, 'Sheet1')
    data = pd.concat([data, data_], ignore_index=True)
data = data.drop(columns='Unnamed: 0',	axis = 1)

In [27]:
len(data)-len(data.drop_duplicates())

24024

In [28]:
data = data.drop_duplicates()

In [31]:
data.to_csv('prosper.csv')

In [30]:
data.columns

Index(['MD', 'TVD', 'PRESSURE', 'TEMPERATURE', 'HOLDUP', 'REGIME',
       'SF_LIQUID_VELOCITY', 'SF_GAS_VELOCITY', 'DIAMETER', 'ROUGHNESS',
       'ANGLE_FROM_VERTICAL', 'TOTAL_MASS_FLOW_RATE', 'LIQUID_MASS_FLOW_RATE',
       'OIL_MASS_FLOW_RATE', 'WATER_MASS_FLOW_RATE', 'GAS_MASS_FLOW_RATE',
       'LIQUID_RATE', 'GAS_RATE', 'WATER_RATE', 'OIL_RATE', 'WCT', 'GOR',
       'OIL_DENSITY', 'GAS_DENSITY', 'LIQUID_DENSITY', 'MIXTURE_DENSITY',
       'OIL_VISCOSITY', 'GAS_VISCOSITY', 'LIQ_VISCOSITY',
       'GAS_LIQ_INTERFACIAL_TENSION', 'GAS_OIL_INTERFACIAL_TENSION',
       'WATER_DENSITY', 'GAS_WAT_INTERFACIAL_TENSION',
       'OIL_WAT_INTERFACIAL_TENSION', 'FRICTION_GRADIENT', 'GRAVITY_GRADIENT',
       'SLIP_LIQ_VEL', 'SLIP_GAS_VEL', 'FREE_GAS_RATE', 'WAT_VISCOSITY',
       'FVF_OIL', 'FVF_WAT', 'FVF_GAS'],
      dtype='object')

In [2]:
prosper = pd.read_csv('prosper.csv')
prosper.drop('Unnamed: 0', axis=1, inplace = True)

In [6]:
prosper

,MD,TVD,PRESSURE,TEMPERATURE,HOLDUP,REGIME,SF_LIQUID_VELOCITY,SF_GAS_VELOCITY,DIAMETER,ROUGHNESS,...,OIL_WAT_INTERFACIAL_TENSION,FRICTION_GRADIENT,GRAVITY_GRADIENT,SLIP_LIQ_VEL,SLIP_GAS_VEL,FREE_GAS_RATE,WAT_VISCOSITY,FVF_OIL,FVF_WAT,FVF_GAS
0,114.3,114.3,63.9359,20.4572,1.0,1,19.378800,0.0,0.062,0.000015,...,3.400000e+35,4.613790e-01,0.082031,19.378800,0.0,0.0,3.400000e+35,1.010910e+00,3.400000e+35,3.400000e+35
1,190.5,190.5,105.4350,20.7620,1.0,1,19.371500,0.0,0.062,0.000015,...,3.400000e+35,4.637540e-01,0.082062,19.371500,0.0,0.0,3.400000e+35,1.010530e+00,3.400000e+35,3.400000e+35
2,266.7,266.7,147.1180,21.0668,1.0,1,19.364100,0.0,0.062,0.000015,...,3.400000e+35,4.661240e-01,0.082094,19.364100,0.0,0.0,3.400000e+35,1.010140e+00,3.400000e+35,3.400000e+35
3,342.9,342.9,188.9840,21.3716,1.0,1,19.356700,0.0,0.062,0.000015,...,3.400000e+35,4.684880e-01,0.082125,19.356700,0.0,0.0,3.400000e+35,1.009750e+00,3.400000e+35,3.400000e+35
4,419.1,419.1,231.0310,21.6764,1.0,1,19.349200,0.0,0.062,0.000015,...,3.400000e+35,4.708430e-01,0.082157,19.349200,0.0,0.0,3.400000e+35,1.009360e+00,3.400000e+35,3.400000e+35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
233371,723.9,723.9,246.5010,22.8956,1.0,1,0.007595,0.0,0.062,0.000015,...,3.400000e+35,6.537950e-07,0.098888,0.007595,0.0,0.0,1.034260e+00,3.400000e+35,9.904900e-01,3.400000e+35
233372,800.1,800.1,254.0370,23.2004,1.0,1,0.007593,0.0,0.062,0.000015,...,3.400000e+35,6.492870e-07,0.098912,0.007593,0.0,0.0,1.027390e+00,3.400000e+35,9.902460e-01,3.400000e+35
233373,876.3,876.3,261.5750,23.5052,1.0,1,0.007591,0.0,0.062,0.000015,...,3.400000e+35,6.448240e-07,0.098936,0.007591,0.0,0.0,1.020580e+00,3.400000e+35,9.900030e-01,3.400000e+35
233374,952.5,952.5,269.1150,23.8100,1.0,1,0.007589,0.0,0.062,0.000015,...,3.400000e+35,6.404060e-07,0.098960,0.007589,0.0,0.0,1.013830e+00,3.400000e+35,9.897610e-01,3.400000e+35


In [5]:
prosper.shape

(233376, 43)

В данном датасете очень много значений 3.400000e+35. Они означают невалидные измерения. Мы можем их либо дропнуть, либо занулить, чтобы они приняли участие в обучении. Попробуем понять, сколько их.

In [15]:
prosper[prosper != 3.400000e+35].dropna().shape

(108958, 43)

Невалидных измерений более половины. Предлагается опробовать обе схемы: обнуление и удаление строк. Начнем с зануления.

In [19]:
data_0 = prosper.replace(3.400000e+35,0)
data_0 = data_0[['OIL_MASS_FLOW_RATE', 
                 'WATER_MASS_FLOW_RATE', 
                 'GAS_MASS_FLOW_RATE',
                 'WCT', 
                 'OIL_DENSITY', 
                 'WATER_DENSITY', 
                 'GAS_DENSITY',
                 'GAS_LIQ_INTERFACIAL_TENSION', 
                 'OIL_VISCOSITY',
                 'WAT_VISCOSITY',
                 'GAS_VISCOSITY',
                 'ROUGHNESS',
                 'DIAMETER',
                 'PRESSURE',
                 'HOLDUP',
                 'FRICTION_GRADIENT',
                 'GRAVITY_GRADIENT']]

In [23]:
data_0.max()

OIL_MASS_FLOW_RATE             1.487300e+07
WATER_MASS_FLOW_RATE           1.000110e+07
GAS_MASS_FLOW_RATE             4.642800e+07
WCT                            1.000000e+00
OIL_DENSITY                    8.607370e+02
WATER_DENSITY                  1.175970e+03
GAS_DENSITY                    6.266700e+02
GAS_LIQ_INTERFACIAL_TENSION    2.958300e+01
OIL_VISCOSITY                  2.309160e+01
WAT_VISCOSITY                  1.091820e+00
GAS_VISCOSITY                  2.223000e-01
ROUGHNESS                      1.524000e-05
DIAMETER                       6.200000e-02
PRESSURE                       6.922650e+04
HOLDUP                         1.000000e+00
FRICTION_GRADIENT              6.659550e+01
GRAVITY_GRADIENT               1.067250e-01
dtype: float64

In [33]:
normalization_matrix = np.diag([1e-7, 1e-7, 1e-8, 1, 1e-3, 1e-3, 1e-3, 1e-2, 1e-2, 1, 1, 1e5, 1e1, 1e-5, 1, 1e-2, 1e1])
mult = data_0.values @ normalization_matrix
mult = pd.DataFrame(mult, columns = ['OIL_MASS_FLOW_RATE', 
                                'WATER_MASS_FLOW_RATE', 
                                'GAS_MASS_FLOW_RATE',
                                'WCT', 
                                'OIL_DENSITY', 
                                'WATER_DENSITY', 
                                'GAS_DENSITY',
                                'GAS_LIQ_INTERFACIAL_TENSION', 
                                'OIL_VISCOSITY',
                                'WAT_VISCOSITY',
                                'GAS_VISCOSITY',
                                'ROUGHNESS',
                                'DIAMETER',
                                'PRESSURE',
                                'HOLDUP',
                                'FRICTION_GRADIENT',
                                'GRAVITY_GRADIENT'])

In [34]:
mult.max()

OIL_MASS_FLOW_RATE             1.487300
WATER_MASS_FLOW_RATE           1.000110
GAS_MASS_FLOW_RATE             0.464280
WCT                            1.000000
OIL_DENSITY                    0.860737
WATER_DENSITY                  1.175970
GAS_DENSITY                    0.626670
GAS_LIQ_INTERFACIAL_TENSION    0.295830
OIL_VISCOSITY                  0.230916
WAT_VISCOSITY                  1.091820
GAS_VISCOSITY                  0.222300
ROUGHNESS                      1.524000
DIAMETER                       0.620000
PRESSURE                       0.692265
HOLDUP                         1.000000
FRICTION_GRADIENT              0.665955
GRAVITY_GRADIENT               1.067250
dtype: float64

Нормализовали все фичи около 1.

In [37]:
torch.set_default_dtype(torch.float32)
labels_tensor = torch.tensor(np.float32(mult[['HOLDUP', 'FRICTION_GRADIENT', 'GRAVITY_GRADIENT']].values))
data_tensor = torch.tensor(np.float32(mult[['OIL_MASS_FLOW_RATE', 
                                            'WATER_MASS_FLOW_RATE', 
                                            'GAS_MASS_FLOW_RATE',
                                            'WCT', 
                                            'OIL_DENSITY', 
                                            'WATER_DENSITY', 
                                            'GAS_DENSITY',
                                            'GAS_LIQ_INTERFACIAL_TENSION', 
                                            'OIL_VISCOSITY',
                                            'WAT_VISCOSITY',
                                            'GAS_VISCOSITY',
                                            'ROUGHNESS',
                                            'DIAMETER',
                                            'PRESSURE']].values))
dataset = create_dataset_from_data(data_tensor, labels_tensor)

In [ ]:
model = KAN([14,42,84,84,3], grid=10, k=5, seed=0)
model.fit(dataset, steps = 20)